In [ ]:
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git


  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-3qxlobgx
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-3qxlobgx
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit fb2becacbf3c48249ce960f3ef60a862f8fc6707
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-9tyxw4k4/elegantrl_9ccb103ec54b4c9e91e47a82dc65a589
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-9tyxw4k4/elegantrl_9ccb103ec54b4c9e91e47a82dc65a589
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit b2ec84b19a82e6f18b781743d8bf37ca27134238
  Preparing metadata (setup.py) ... done


In [ ]:
import itertools

import pandas as pd
from stable_baselines3.common.logger import configure

from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.config import DATA_SAVE_DIR
from finrl.config import INDICATORS
from finrl.config import RESULTS_DIR
from finrl.config import TENSORBOARD_LOG_DIR
from finrl.config import TEST_END_DATE
from finrl.config import TEST_START_DATE
from finrl.config import TRAINED_MODEL_DIR
from finrl.config_tickers import DOW_30_TICKER
from finrl.main import check_and_make_directories
from finrl.meta.data_processors.func import calc_train_trade_data
from finrl.meta.data_processors.func import calc_train_trade_starts_ends_if_rolling
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.meta.preprocessor.preprocessors import data_split
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.plot import backtest_stats
from finrl.plot import get_baseline
from finrl.plot import plot_return
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def stock_trading_rolling_window(
    rolling_window_length: int,
    if_store_actions: bool = True,
    if_store_result: bool = True,
    if_using_a2c: bool = True,
    if_using_ddpg: bool = True,
    if_using_ppo: bool = True,
    if_using_sac: bool = True,
    if_using_td3: bool = True,):
  init_train_data = pd.read_csv('/content/drive/MyDrive/SympactAI_Testing_Data/train_data.csv')
  init_test_data = pd.read_csv('/content/drive/MyDrive/SympactAI_Testing_Data/test_data.csv')

  stock_dimension = len(init_train_data.tic.unique())
  state_space = 1 + 2 * stock_dimension + len(INDICATORS) * stock_dimension
  print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
  buy_cost_list = sell_cost_list = [0.001] * stock_dimension
  num_stock_shares = [0] * stock_dimension

  initial_amount = 1000000
  env_kwargs = {
      "hmax": 100,
      "initial_amount": initial_amount,
      "num_stock_shares": num_stock_shares,
      "buy_cost_pct": buy_cost_list,
      "sell_cost_pct": sell_cost_list,
      "state_space": state_space,
      "stock_dim": stock_dimension,
      "tech_indicator_list": INDICATORS,
      "action_space": stock_dimension,
      "reward_scaling": 1e-4,
  }

  # split the init_train_data and init_trade_data to subsets
  init_train_dates = init_train_data["date"].unique()
  init_test_dates = init_test_data["date"].unique()

  (
      train_starts,
      train_ends,
      trade_starts,
      trade_ends,
  ) = calc_train_trade_starts_ends_if_rolling(
      init_train_dates, init_test_dates, rolling_window_length
  )

  result = pd.DataFrame()
  # actions_a2c = pd.DataFrame(columns=DOW_30_TICKER)
  actions_ddpg = pd.DataFrame(columns=DOW_30_TICKER)

  import time

  total_iterations = len(train_starts)
  start_time_overall = time.time()

  for i in range(len(train_starts)):
        print("i: ", i)
        iter_start_time = time.time()

        print(f"\n--- Iteration {i+1}/{total_iterations} ---")

        train_data, trade_data = calc_train_trade_data(
            i,
            train_starts,
            train_ends,
            trade_starts,
            trade_ends,
            init_train_data,
            init_test_data,
            "date",
        )
        e_train_gym = StockTradingEnv(df=train_data, **env_kwargs)
        env_train, _ = e_train_gym.get_sb_env()


        if if_using_ddpg:
              if len(result) >= 1:
                  env_kwargs["initial_amount"] = result["DDPG"].iloc[-1]
              e_train_gym = StockTradingEnv(df=train_data, **env_kwargs)
              env_train, _ = e_train_gym.get_sb_env()
              agent = DRLAgent(env=env_train)
              model_ddpg = agent.get_model("ddpg")
              # set up logger
              tmp_path = "/content/drive/MyDrive/SympactAI_Testing_Data/ddpg_rw_logs"
              new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
              # Set new logger
              model_ddpg.set_logger(new_logger_ddpg)
              trained_ddpg = agent.train_model(
                  model=model_ddpg, tb_log_name="ddpg", total_timesteps=1000
              )
              trained_ddpg.save("/content/drive/MyDrive/SympactAI_Testing_Data/trained_models/agent_ddpg_rolling_window.zip")

              #  Time tracking
              iter_end_time = time.time()
              elapsed_time = iter_end_time - iter_start_time
              avg_time_per_iter = (iter_end_time - start_time_overall) / (i + 1)
              remaining_iters = total_iterations - (i + 1)
              est_remaining_time = remaining_iters * avg_time_per_iter

              def format_time(seconds):
                  m, s = divmod(int(seconds), 60)
                  h, m = divmod(m, 60)
                  return f"{h}h {m}m {s}s"

              print(f"Iteration time: {format_time(elapsed_time)}")
              print(f"Estimated remaining time: {format_time(est_remaining_time)}")



        # trade
        # this e_trade_gym is initialized, then it will be used if i == 0
        e_trade_gym = StockTradingEnv(
            df=trade_data,
            turbulence_threshold=70,
            risk_indicator_col="vix",
            **env_kwargs,
        )


        if if_using_ddpg:
            if len(result) >= 1:
                env_kwargs["initial_amount"] = result["DDPG"].iloc[-1]
                e_trade_gym = StockTradingEnv(
                    df=trade_data,
                    turbulence_threshold=70,
                    risk_indicator_col="vix",
                    **env_kwargs,
                )
            result_ddpg, actions_i_ddpg = DRLAgent.DRL_prediction(
                model=trained_ddpg, environment=e_trade_gym
            )


        if if_using_ddpg and isinstance(result_ddpg, tuple):
            actions_i_ddpg = result_ddpg[1]
            result_ddpg = result_ddpg[0]


        # merge actions
        actions_ddpg = (
            pd.concat([actions_ddpg, actions_i_ddpg]) if if_using_ddpg else None
        )


        # dji_i
        trade_start = trade_starts[i]
        trade_end = trade_ends[i]
        dji_i_ = get_baseline(ticker="^DJI", start=trade_start, end=trade_end)
        dji_i = pd.DataFrame()
        dji_i["date"] = dji_i_["date"]
        dji_i["DJI"] = dji_i_["close"]
        # dji_i.rename(columns={'account_value': 'DJI'}, inplace=True)

        # select the rows between trade_start and trade_end (not included), since some values may not in this region
        dji_i = dji_i.loc[
            (dji_i["date"] >= trade_start) & (dji_i["date"] < trade_end)
        ]

        # init result_i by dji_i
        result_i = dji_i
        # rename column name of result_a2c, result_ddpg, etc., and then put them to result_i

        if if_using_ddpg:
            result_ddpg.rename(columns={"account_value": "DDPG"}, inplace=True)
            result_i = pd.merge(result_i, result_ddpg, how="left")

        # remove the rows with nan
        result_i = result_i.dropna(axis=0, how="any")

        # merge result_i to result
        result = pd.concat([result, result_i], axis=0)
  # store actions
  if if_store_actions:
        actions_ddpg.to_csv("/content/drive/MyDrive/SympactAI_Testing_Data/ddpg_rw_results_actions/actions_ddpg_rw.csv") if if_using_ddpg else None

    # calc the column name of strategies, including DJI
  col_strategies = []
  for col in result.columns:
      if col != "date" and col != "" and "Unnamed" not in col:
          col_strategies.append(col)

    # make sure that the first row of DJI is initial_amount
  col = "DJI"
  result[col] = result[col] / result[col].iloc[0] * initial_amount
  result = result.reset_index(drop=True)
 # stats
  for col in col_strategies:
      stats = backtest_stats(result, value_col_name=col)
      print("\nstats of " + col + ": \n", stats)

    # print and save result
  print("result: ", result)
  if if_store_result:
      result.to_csv("/content/drive/MyDrive/SympactAI_Testing_Data/ddpg_rw_results_actions/result_ddpg_rw.csv")
 # plot fig
  plot_return(
      result=result,
      column_as_x="date",
      if_need_calc_return=True,
      savefig_filename="/content/drive/MyDrive/SympactAI_Testing_Data/plots/ddpg_rw_vs_dji.png",
      xlabel="Date",
      ylabel="Return",
      if_transfer_date=True,
      num_days_xticks=20,
  )



In [ ]:
rolling_window_length = 63  # num of trading days in a rolling window
if_store_actions = True
if_store_result = True

if_using_ddpg = True

stock_trading_rolling_window(
    rolling_window_length=rolling_window_length,
    if_store_actions=if_store_actions,
    if_store_result=if_store_result,
    if_using_ddpg=if_using_ddpg,
)

Stock Dimension: 29, State Space: 291
num_subsets_if_rolling:  4
train_starts:  ['2010-01-04', '2010-04-06', '2010-07-06', '2010-10-04']
train_ends__:  ['2024-07-01', '2024-09-30', '2024-12-30', '2025-04-02']
trade_starts:  ['2024-07-01', '2024-09-30', '2024-12-30', '2025-04-02']
trade_ends__:  ['2024-09-30', '2024-12-30', '2025-04-02', '2025-06-27']
i:  0

--- Iteration 1/4 ---
{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device
Logging to /content/drive/MyDrive/SympactAI_Testing_Data/ddpg_rw_logs
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'r

[*********************100%***********************]  1 of 1 completed

Iteration time: 0h 0m 42s
Estimated remaining time: 0h 2m 6s
hit end!
Shape of DataFrame:  (63, 8)
i:  1

--- Iteration 2/4 ---
{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device
Logging to /content/drive/MyDrive/SympactAI_Testing_Data/ddpg_rw_logs


Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging 

[*********************100%***********************]  1 of 1 completed

...

Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging 

[*********************100%***********************]  1 of 1 completed

hit end!
Shape of DataFrame:  (59, 8)
Annual return          0.109497
Cumulative returns     0.107668
Annual volatility      0.175932
Sharpe ratio           0.680303
Calmar ratio           0.668919
Stability              0.057283
Max drawdown          -0.163692
Omega ratio            1.138649
Sortino ratio          1.011307
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.863333
Daily value at risk   -0.021690
dtype: float64

stats of DJI: 
 Annual return          0.109497
Cumulative returns     0.107668
Annual volatility      0.175932
Sharpe ratio           0.680303
Calmar ratio           0.668919
Stability              0.057283
Max drawdown          -0.163692
Omega ratio            1.138649
Sortino ratio          1.011307
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.863333
Daily value at risk   -0.021690
dtype: float64
Annual return          0.305635
Cumulative returns     0.300120
Annual volatility  


/usr/local/lib/python3.11/dist-packages/finrl/plot.py:316: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, row], axis=0)


final return:  {'DJI': np.float64(0.10766842082745898), 'DDPG': np.float64(0.30011953737421915)}
